In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import sys
import pickle
import tensorflow as tf
from sklearn.utils import shuffle
import tsc_utils as tu
import time
import datetime

In [3]:
model_list = [
    dict(model = tu.lenet, model_params = tu.params_orig_lenet),
    dict(model = tu.lenet, model_params = tu.params_big_lenet),
    dict(model = tu.sermanet, model_params = tu.params_sermanet),
    dict(model = tu.sermanet, model_params = tu.params_sermanet_big),    
    dict(model = tu.sermanet_v2, model_params = tu.params_sermanet_v2),
    dict(model = tu.sermanet_v2, model_params = tu.params_sermanet_v2_big)
]


for m in model_list:
    g = tf.Graph()
    with g.as_default():
        model = m['model']
        model_params = m['model_params']
        x = tf.placeholder(tf.float32, (None, 32, 32, 1))
        y = tf.placeholder(tf.int32, (None))
        one_hot_y = tf.one_hot(y, model_params['num_classes'])
        is_training = tf.placeholder(tf.bool)

        logits = model(x, params=model_params, is_training=is_training)

    with tf.Session(graph=g) as sess:
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            print(variable)
            print(shape)
            #print(len(shape))
            variable_parametes = 1
            for dim in shape:
                #print(dim)
                variable_parametes *= dim.value
            #print(variable_parametes)
            total_parameters += variable_parametes
        print(total_parameters)
        m['total_params'] = total_parameters

{'name': 'orig_lenet', 'num_classes': 43, 'fc3_size': 120, 'model_name': 'lenet', 'fc4_size': 84, 'conv1_p': 0.95, 'conv2_k': 5, 'conv2_d': 16, 'conv2_p': 0.95, 'fc4_p': 0.5, 'fc3_p': 0.5, 'conv1_d': 6, 'conv1_k': 5}
lenet pool2 reshaped size:  [None, 1024]
<tf.Variable 'conv1/weights:0' shape=(5, 5, 1, 6) dtype=float32_ref>
(5, 5, 1, 6)
<tf.Variable 'conv1/biases:0' shape=(6,) dtype=float32_ref>
(6,)
<tf.Variable 'conv2/weights:0' shape=(5, 5, 6, 16) dtype=float32_ref>
(5, 5, 6, 16)
<tf.Variable 'conv2/biases:0' shape=(16,) dtype=float32_ref>
(16,)
<tf.Variable 'fc3/weights:0' shape=(1024, 120) dtype=float32_ref>
(1024, 120)
<tf.Variable 'fc3/biases:0' shape=(120,) dtype=float32_ref>
(120,)
<tf.Variable 'fc4/weights:0' shape=(120, 84) dtype=float32_ref>
(120, 84)
<tf.Variable 'fc4/biases:0' shape=(84,) dtype=float32_ref>
(84,)
<tf.Variable 'out/weights:0' shape=(84, 43) dtype=float32_ref>
(84, 43)
<tf.Variable 'out/biases:0' shape=(43,) dtype=float32_ref>
(43,)
139391
{'name': 'big_le

In [7]:
for m in model_list:
    print(m['model'].__name__, m['model_params']['name'], m['total_params'])

lenet orig_lenet 139391
lenet big_lenet 2137739
sermanet standard 1681359
sermanet big 4351643
sermanet_v2 standard 3972139
sermanet_v2 big 15796267
